In [1]:
import selenium
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver import ActionChains

from bs4 import BeautifulSoup
import requests

In [2]:
driver = webdriver.Chrome(r"C:\Users\vinay\OneDrive\Desktop\FlipRobo\Selenium\chromedriver_win32\chromedriver.exe")
time.sleep(6)

# 1)

In [6]:
url = 'https://www.amazon.in'
driver.get(url)
time.sleep(6)

# inserting search word
search_but = driver.find_element_by_id('twotabsearchtextbox')
search_but.send_keys('Guitar')
time.sleep(2)

# clicking search button
send_but = driver.find_element_by_id('nav-search-submit-button')
send_but.click()
time.sleep(3)

# 2)

In the above question, now scrape the following details of each product listed in first 3 pages 
of your search results and save it in a dataframe and csv. In case if any product vertical has 
less than 3 pages in search results then scrape all the products available under that product 
vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of 
Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" 
and “Product URL”. In case, if any of the details are missing for any of the product then 
replace it by “-“

In [7]:
#First we will get all the product links from all the 3 pages and iterate over them.

links = []


to_get = driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-4']/a")
for i in to_get:
    links.append(i.get_attribute('href'))
time.sleep(5)

#clicking on the next page to move to page 2
ur1_next2 =driver.find_element_by_xpath("//li[@class='a-last']/a")
ur1_next2.click()
time.sleep(8)

# Extracting the product links 
to_get1 = driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-4']/a")
for i in to_get1:
    links.append(i.get_attribute('href'))
time.sleep(5)

# clickiing on next page to move to page 3
ur1_next3 =driver.find_element_by_xpath("//li[@class='a-last']/a")
ur1_next3.click()
time.sleep(8)

# Extracting the product links 
to_get2 = driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-4']/a")
for i in to_get2:
    links.append(i.get_attribute('href'))
time.sleep(5)

len(links)

brand = []
name = []
rating = []
no_rating = []
price =[]
retur = []
delvery = []
avail = []
other= []
product_url=[]

for i in links:
    driver.get(i)
    time.sleep(5)

    #Name of the products
    names = driver.find_element_by_xpath("//span[@id='productTitle']")
    name.append(names.text)
    time.sleep(1)
    
    #Brand name
    try:
        brand_name = driver.find_element_by_xpath("//div[@id='merchant-info']/a[1]")
        brand.append(brand_name.text)
    except:
        brand.append("-")
    time.sleep(1)

    #Rating
    try:
        rat = driver.find_element_by_xpath("//span[@data-hook='rating-out-of-text']")
        rating.append(rat.text)
    except:
        rating.append("-")
    time.sleep(1)

    #Number of rating
    try:
        no_rate = driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']")
        no_rating.append(no_rate.text)
    except:
        no_rating.append("-")
    time.sleep(1)

    #Price of the product
    try:
        amt = driver.find_element_by_xpath("//span[@class='a-size-medium a-color-price priceBlockBuyingPriceString']")
        price.append(amt.text)
    except:
        price.append("-")
    time.sleep(1)

    #Delivery date
    try:
        day = driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b")
        delvery.append(day.text)
    except:
        delvery.append("-")
    time.sleep(1)

    #Item availability
    try:
        avail_1 = driver.find_element_by_xpath("//span[@class='a-size-medium a-color-success']")
        avail.append(avail_1.text)
    except:
        avail.append("-")
    time.sleep(1)

    #Other details
    try:
        desc = driver.find_element_by_xpath("//ul[@class='a-unordered-list a-vertical a-spacing-mini']/li[1]/span")
        other.append((desc.text).replace('"'," "))
    except:
        other.append("-")
    time.sleep(1)

    #Return/Exchange
    try:
        exchange = driver.find_element_by_xpath("//a[@class='a-size-small a-link-normal a-text-normal']")
        retur.append(exchange.text)
    except:
        retur.append("-")
    time.sleep(1)

#Creating a dataframe.
guitar = pd.DataFrame({})

guitar['Brand'] = brand
guitar['Name of the Product'] = name
guitar['Rating'] = rating
guitar['No. of rating'] = no_rating
guitar['Price'] = price
guitar['Return/Exchange'] = retur
guitar['Expected Delivery'] = delvery
guitar['Availability'] = avail
guitar['Other Details'] = other
guitar['Product URL'] = links

guitar

,Brand,Name of the Product,Rating,No. of rating,Price,Return/Exchange,Expected Delivery,Availability,Other Details,Product URL
0,NoteKart,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,4 out of 5,446 ratings,"₹ 6,999.00",Pay on Delivery,June 18 - 22,-,QUALITY STRINGS ---The surface is coated with ...,https://www.amazon.in/gp/slredirect/picassoRed...
1,NoteKart,Kadence Slowhand Premium Jumbo Size Semi Acous...,4.3 out of 5,128 ratings,-,Pay on Delivery,"Sunday, June 13",In stock.,-,https://www.amazon.in/gp/slredirect/picassoRed...
2,Appario Retail Private Ltd,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",3.9 out of 5,"9,683 ratings","₹ 2,499.00",Pay on Delivery,"Sunday, June 13",In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/gp/slredirect/picassoRed...
3,Appario Retail Private Ltd,JUAREZ 38 Inch Acoustic Guitar with Dual Actio...,3 out of 5,20 ratings,-,Pay on Delivery,"Sunday, June 13",In stock.,"Glossy finish, Number of frets - 18, 38 Inch A...",https://www.amazon.in/gp/slredirect/picassoRed...
4,Appario Retail Private Ltd,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",3.9 out of 5,"9,683 ratings","₹ 2,499.00",Pay on Delivery,"Sunday, June 13",In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
169,TYRUS ONE,TYRUS ONE� Guitar 4-String Acoustic Guitar Mus...,-,-,₹ 698.00,7 Days Returnable,June 10 - 11,In stock.,Very Crisp and Clear Sound of Strings,https://www.amazon.in/TYRUS-4-String-Acoustic-...
170,Appario Retail Private Ltd,"JUAREZ Arpeggio 39 Inch Acoustic Guitar Kit, S...",4.1 out of 5,6 ratings,"₹ 5,202.00",Pay on Delivery,"Sunday, June 13",In stock.,"Juarez Arpeggio 39 Inch Acoustic Guitar Kit, S...",https://www.amazon.in/JUAREZ-Arpeggio-Acoustic...
171,KB_CREATION,"KB 4-string acoustic guitar learning kids toy,...",-,-,₹ 899.00,7 Days Replacement,"Thursday, June 10",Only 2 left in stock.,Very Crisp and Clear Sound of Strings,https://www.amazon.in/KB-4-string-acoustic-lea...
172,Appario Retail Private Ltd,JUAREZ Fiésta 41 Inch Acoustic Guitar Cutaway ...,3.7 out of 5,25 ratings,"₹ 4,009.00",Pay on Delivery,"Sunday, June 13",In stock.,"Glossy finish, Number of frets - 21, 41 Inch A...",https://www.amazon.in/JUAREZ-Acoustic-Cutaway-...


# 3)

In [10]:
url = 'https://images.google.com/'
driver.get(url) # Entering url
time.sleep(3)

search = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
search.send_keys('fruits') #Entering fruits in search bar.

search_but = driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']")
search_but.click()
time.sleep(3)

# 500 time we scroll down by 100000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,100000)")
time.sleep(6)
    
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]') ##Retreiving all the image links

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):    ### segregating link and saving links which start with 'http'
            img_urls.append(source)
time.sleep(3)           
            
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\vinay\OneDrive\Desktop\FlipRobo\Selenium\Scrape image"+str(i)+".jpg", "wb")
    file.write(response.content) ##Saving the image files

# 4)
Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on 
www.flipkart.com and scrape following details for all the search results displayed on 1st page. 
Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, 
“Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display 
Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. 
Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe 
and CSV.


In [11]:
url = 'https://www.flipkart.com'
driver.get(url)
time.sleep(6)

#Clicking cross button.
try:
    cross_but = driver.find_element_by_xpath("//button[@class ='_2KpZ6l _2doB4z']")
    cross_but.click()
    time.sleep(3)
except:
    pass

# finding element for job search bar
search_item = driver.find_element_by_class_name('_3704LK')
search_item.clear()

search_item.send_keys('pixel 4A')
time.sleep(3)

#Clicking search button.
search_but = driver.find_element_by_xpath("//button[@class ='L0Z3Pu']")
search_but.click()
time.sleep(3)

brand = []
phone = []
colour =[]
ram = []
rom = []
pri_camera = []
sec_camera = []
size = []
resolu = []
process = []
process_core = []
battery = []
price = []
links = []

## links of all the first page dispayed products.
to_get = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
for i in to_get:
    links.append(i.get_attribute('href'))
time.sleep(5)

##Brand names and phone names.
brnd = driver.find_elements_by_xpath("//div[@class='_4rR01T']")
for i in brnd:
    brand.append((i.text).split(' ')[0])
time.sleep(2)

## battery
battery_all = driver.find_elements_by_xpath("//div[@class='fMghEO']/ul/li[4]")
for i in battery_all:
    battery.append(i.text)
time.sleep(2)

for i in links:
    driver.get(i)
    time.sleep(4)
    
    #clicking read more button
    read = driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']")
    read.click()
    
    #model name
    model = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][1]/table/tbody/tr[3]/td[2]")
    phone.append(model.text)
    
    
    #color
    color = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][1]/table/tbody/tr[4]/td[2]")
    colour.append(color.text)
    
    #ram and rom
    space = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][4]/table/tbody/tr[1]")
    rom.append(space.text)
    
    speed = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][4]/table/tbody/tr[2]")
    ram.append(speed.text)
    
    #camera
    try:
        primary_camera = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody/tr[2]/td[2]")
        pri_camera.append(primary_camera.text)
    except:
        primary_camera = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody/tr/td[2]")
        pri_camera.append(primary_camera.text)
        
    try:
        second_camera = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody/tr[5]/td[2]")
        sec_camera.append(second_camera.text)
    except:
        sec_camera.append("--")
        
    #Display and resolution
    display_size = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][2]/table/tbody/tr[1]/td[2]")
    size.append(display_size.text)
    
    display_reso = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][2]/table/tbody/tr[2]/td[2]")
    resolu.append((display_reso.text).replace("$"," "))
    
    #processor and processor cores
    processor = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table/tbody/tr[2]/td[2]")
    process.append(processor.text)
    
    processor_type = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table/tbody/tr[3]/td[2]")
    process_core.append(processor_type.text)
    
    #Price
    amt = driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
    price.append(amt.text)
    
    time.sleep(3)

#Creating a dataframe.
flipkart_product = pd.DataFrame({})

flipkart_product['Brand'] = brand
flipkart_product['Name of the Product'] = phone
flipkart_product['Color'] = colour
flipkart_product['Price'] = price
flipkart_product['RAM'] = ram
flipkart_product['ROM'] = rom
flipkart_product['Primary_camera'] = pri_camera
flipkart_product['Secondary_camera'] = sec_camera
flipkart_product['Display_size'] = size
flipkart_product['Display_resolution'] = resolu
flipkart_product['Processor'] = process
flipkart_product['Processor_core'] = process_core
flipkart_product['Battery'] = battery
flipkart_product['Product URL'] = links

flipkart_product

,Brand,Name of the Product,Color,Price,RAM,ROM,Primary_camera,Secondary_camera,Display_size,Display_resolution,Processor,Processor_core,Battery,Product URL
0,Google,Pixel 4a,Just Black,"₹29,999",RAM\n6 GB,Internal Storage\n128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 730G,Octa Core,3140 mAh Battery,https://www.flipkart.com/google-pixel-4a-just-...
1,REDMI,Note 9,Pebble Grey,"₹12,499",RAM\n4 GB,Internal Storage\n64 GB,48MP + 8MP + 2MP + 2MP,Yes,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh Battery,https://www.flipkart.com/redmi-note-9-pebble-g...
2,REDMI,9 Power,Electric Green,"₹10,499",RAM\n4 GB,Internal Storage\n64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh Battery,https://www.flipkart.com/redmi-9-power-electri...
3,REDMI,9 Power,Mighty Black,"₹10,499",RAM\n4 GB,Internal Storage\n64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh Battery,https://www.flipkart.com/redmi-9-power-mighty-...
4,REDMI,9 Power,Mighty Black,"₹12,999",RAM\n6 GB,Internal Storage\n128 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 pixel,Qualcomm® Snapdragon™ 662,Octa Core,6000 mAh Battery,https://www.flipkart.com/redmi-9-power-mighty-...
5,REDMI,9 Power,Fiery Red,"₹10,499",RAM\n4 GB,Internal Storage\n64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh Battery,https://www.flipkart.com/redmi-9-power-fiery-r...
6,REDMI,Note 9,Aqua Green,"₹12,990",RAM\n4 GB,Internal Storage\n64 GB,48MP + 8MP + 2MP + 2MP,Yes,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh Battery,https://www.flipkart.com/redmi-note-9-aqua-gre...
7,REDMI,9 Power,Blazing Blue,"₹10,499",RAM\n4 GB,Internal Storage\n64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh Battery,https://www.flipkart.com/redmi-9-power-blazing...
8,Redmi,Redmi 9,Sporty Orange,"₹9,236",RAM\n4 GB,Internal Storage\n64 GB,13MP + 8MP,5MP Front Camera,16.59 cm (6.53 inch),720 x 1600 pixel,MediaTek Helio G35,2.3 GHz,5000 mAh Battery,https://www.flipkart.com/redmi-9-sporty-orange...
9,REDMI,9 Prime,Matte Black,"₹9,999",RAM\n4 GB,Internal Storage\n64 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh Battery,https://www.flipkart.com/redmi-9-prime-matte-b...


# 5)

Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on 
google maps.

In [12]:
url = "https://www.google.com/maps/"
driver.get(url)
time.sleep(3)

#Search box inputing city name
search_space = driver.find_element_by_xpath("//input[@autofocus='autofocus']")
search_space.send_keys("New Delhi")
time.sleep(1)

#Clicking search button
search_button = driver.find_element_by_xpath("//button[@id='searchbox-searchbutton']")
search_button.click()
time.sleep(3)

#Extracting geo location from url link.
url_now = driver.current_url       #Gives current URL

url1 = url_now.split('@')
url2 = url1[1].split(',')          # Spliting url for the geo codes

Latitude = []
Longitude = []

Latitude.append(url2[0])
Longitude.append(url2[1])

print(Latitude,Longitude )         # Printing latitude and longitude

['16.3859047'] ['77.9395859']


# 6)
Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 –
September 20) from trak.in.

In [13]:
url = 'https://trak.in/'
driver.get(url)
time.sleep(6)

driver.maximize_window()  ## Maximizing the window to make sure our target button is visible

fund = driver.find_element_by_xpath("//li[@id = 'menu-item-51510']")
fund.click()
time.sleep(4)

driver.refresh()
time.sleep(3)

fund = driver.find_element_by_xpath("//li[@id = 'menu-item-51510']")
fund.click()
time.sleep(5)

date=[]
name = []
vertical = []
sub_vertical=[]
city=[]
investor=[]
investor_type=[]
amount=[]
time.sleep(3)

########## JULY
dates_july = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[2]")
for i in dates_july:
    date.append(i.text)  ##extracting dates from july

names_july = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[3]")
for i in names_july:
    name.append(i.text) ## extracting startup names from july

vert_july = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[4]")
for i in vert_july:
    vertical.append(i.text)  ## extracting industry names from july

sub_vert_july = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[5]")
for i in sub_vert_july:
    sub_vertical.append(i.text)  ## extracting sub-industry names from july
     
city_july = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[6]")
for i in city_july:
    city.append(i.text)  ## extracting city names from july

invest_july = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[7]")
for i in invest_july:
    investor.append(i.text) ## extracting investor's name names from july


invest_type_july = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[8]")
for i in invest_type_july:
    investor_type.append(i.text)  ## extracting investor type names from july

amount_july = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[9]")
for i in amount_july:
    amount.append(i.text)  ## extracting Amount names from july
    
time.sleep(3)

########## AUGUST
dates_august = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[2]")
for i in dates_august:
    date.append(i.text)  ####extracting dates from august
       
names_august = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[3]")
for i in names_august:
    name.append(i.text) ####extracting startup names from august


vert_august = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[4]")
for i in vert_august:
    vertical.append(i.text)   ####extracting industry names from august

sub_vert_august = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[5]")
for i in sub_vert_august:
    sub_vertical.append(i.text)  ####extracting sub-industry names from august

city_august = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[6]")
for i in city_august:
    city.append(i.text)   ####extracting city names from august
    
    
invest_august = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[7]")
for i in invest_august:
    investor.append(i.text)  ####extracting investors names from august
       
invest_type_august = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[8]")
for i in invest_type_august:
    investor_type.append(i.text)  ####extracting investor types from august
    
    
amount_august = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[9]")
for i in amount_august:
    amount.append(i.text)  ####extracting amount from august
time.sleep(3)

########## SEPTEMBER
dates_september = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[2]")
for i in dates_september:
    date.append(i.text) ####extracting dates from september
    
names_september = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[3]")
for i in names_september:
    name.append(i.text)  #### extracting startup names from september

vert_september = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[4]")
for i in vert_september:
    vertical.append(i.text)  #### extracting industry names from september
    
sub_vert_september = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[5]")
for i in sub_vert_september:
    sub_vertical.append(i.text)  #### extracting sub-industry names from september
    

city_september = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[6]")
for i in city_september:
    city.append(i.text)  #### extracting city names from september
    
    
invest_september = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[7]")
for i in invest_september:
    investor.append(i.text)  #### extracting investors names from september
    
    
invest_type_september = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[8]")
for i in invest_type_september:
    investor_type.append(i.text)  #### extracting investor type from september
    
amount_september = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[9]")
for i in amount_september:
    amount.append(i.text)  #### extracting amount from september
time.sleep(3)

trak = pd.DataFrame({})  ##creating a dataframe.

trak['Date'] = date
trak['Startup_name'] = name
trak['Industry/vertical'] = vertical
trak['Sub_vertical'] = sub_vertical
trak['City/Location'] = city
trak['Investor_name'] = investor
trak['Investor_type'] = investor_type
trak['Amount(In USD)'] = amount

trak

,Date,Startup_name,Industry/vertical,Sub_vertical,City/Location,Investor_name,Investor_type,Amount(In USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


# 7)
Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [14]:
url = 'https://www.digit.in/reviews/'
driver.get(url)
time.sleep(6)

search = driver.find_element_by_xpath("//a[@data-toggle='modal']").click()
time.sleep(1)

input_ = driver.find_element_by_xpath("//input[@id='globalPageSearchText'][1]")
input_.send_keys("Best Gaming Laptops") ## Entering the search bar
time.sleep(3)

from selenium.webdriver.common.keys import Keys

input_.send_keys(Keys.RETURN) #### clicking enter
time.sleep(6)

laptop = []
lap_links=[]

os = []
display = []
processor = []
memory = []
price = []
digit_rating = []

laptop_names = driver.find_elements_by_xpath("//div[@class='searchProduct-desc']")
for i in laptop_names:
    laptop.append(i.text) ##Saving laptop names
time.sleep(3)

laptop_links = driver.find_elements_by_xpath("//div[@class='searchPage']/a")
for i in laptop_links:
    lap_links.append(i.get_attribute('href')) ##Saving laptop links
time.sleep(3)

for i in lap_links:
    driver.get(i)
    time.sleep(6)
    
    ##### Retriving Operating System
    try:
        details_os = driver.find_element_by_xpath("//div[@class='Specs-Wrap  notspecs ']/ul/li[1]/div")
        os.append(details_os.text)
    except:
        os.append("NA")
    time.sleep(2)
    
    
    ##### Retriving Display
    try:
        details_display = driver.find_element_by_xpath("//div[@class='Specs-Wrap  notspecs ']/ul/li[2]/div")
        display.append(details_display.text)
    except:
        display.append("NA")
    time.sleep(2)
    
    ##### Retriving Processor
    try:
        details_process = driver.find_element_by_xpath("//div[@class='Specs-Wrap  notspecs ']/ul/li[3]/div")
        processor.append(details_process.text)
    except:
        processor.append("NA")
    time.sleep(2)
    
    ##### Retriving Memory
    try:
        details_memory = driver.find_element_by_xpath("//div[@class='Specs-Wrap  notspecs ']/ul/li[2]/div")
        memory.append(details_memory.text)
    except:
        memory.append("NA")
    time.sleep(2)
    
    ##### Retriving Price
    try:
        detail_price = driver.find_element_by_xpath("//div[@class='Block-price']/b")
        price.append(detail_price.text)
    except:
        price.append("NA")
    time.sleep(2)
    
    ##### Retriving Rating
    try:
        rate = driver.find_element_by_xpath("//div[@class='score-Bar']/ul/li/p")
        digit_rating.append(rate.text)
    except:
        digit_rating.append("NA")
    time.sleep(2)

digit = pd.DataFrame({})

digit['Laptop Name'] = laptop
digit['Operating System'] = os
digit['Display'] = display
digit['Processor'] = processor
digit['Memory'] = memory
digit['Price'] = price
digit['Digit Rating'] = digit_rating

digit

,Laptop Name,Operating System,Display,Processor,Memory,Price,Digit Rating
0,Asus G750JX-CV069P,NA,NA,NA,NA,"149,999",NA
1,Lenovo G50-30,NA,NA,NA,NA,"25,694",NA
2,ASUS G53J - A 3D gamer's delight but we want more,NA,NA,NA,NA,"112,499",70
3,Dell Vostro 15 3500,Windows 10 Home,"15.6"" (1920 X 1080)",11th Generation Intel® Core™ I3-1115G4 | NA,"15.6"" (1920 X 1080)","38,489",NA
4,Samsung Series 9 (NP-900X3C),NA,NA,NA,NA,NA,90
5,ASUS ROG Flow x13,Windows 10 Pro,"13.4"" (3840 X 2400)",AMD Ryzen™ 9 5980HS Processor 3.1 GHz | NA,"13.4"" (3840 X 2400)","154,990",68
6,Asus GTX980 Matrix Platinum,NA,NA,NA,NA,"52,000",72
7,HP Envy dv6-7206tx notebook,Windows 8 64 Bit,"15.6"" (1366 X 768)",Intel Core I7 (3rd Generation) | 2.4 Ghz,"15.6"" (1366 X 768)","65,705",90
8,HP Envy 15-k006tx,NA,NA,NA,NA,"80,510",NA
9,Lenovo IdeaPad Y570,Windows 7 (64-Bit),"15.6"" (1366 X 768)",Intel Core I7 (2nd Generation) | 2 Ghz,"15.6"" (1366 X 768)","49,900",90
